# Wellbore Paths in PyVista 

This notebook was adapted from a resource found online, but apparently now removed.

This is an attempt to use pyvista and/or pvgeo to visualise borehole log data in the browser. 

compressed data for this notebook is available for download from [this link](https://cloudstor.aarnet.edu.au/plus/s/gTAVfJZ5Dve15ud)


Now we are going to import `pandas` for reading in our well data, `numpy` for making stratigraphic surfaces, `pyvista` and `PVGeo` for visualization. Also we are going to set the `panel.extension` to `'vtk'` and set the `pyvista` theme to `document

In [ ]:
import pandas as pd
import numpy as np
import pyvista as pv
import panel
import PVGeo

panel.extension("vtk")
pv.set_plot_theme("document")
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import sys
import os

ela_from_source = False
ela_from_source = True

if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\Users\SUD011\Documents\pyela-sudhir'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/github_jm/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *

In [ ]:
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'


## Global settings and preprocessed data


In [ ]:
if ('ELA_DATA' in os.environ):
    data_path = os.environ['ELA_DATA']
elif sys.platform == 'win32':
    data_path = r'C:\data\Lithology'
else:
    username = os.environ['USER']
    data_path = os.path.join('/home', username, 'data', 'Lithology')

# to be reused in experimental notebooks:
classified_logs_filename = os.path.join(data_path, 'Bungendore','classified_logs.pkl')
df = pd.read_pickle(classified_logs_filename)
# df = pd.read_pickle(os.path.join(data_path, 'data_inter', 'above_leederville.pkl'))

In [ ]:
import pickle 

bungendore_datadir = os.path.join(data_path, 'Bungendore')
fp = os.path.join(bungendore_datadir, 'dem_array_data.pkl')
with open(fp, 'rb') as handle:
    dem_array_data = pickle.load(handle)

In [ ]:
classified_logs_filename

In [ ]:
len(df)

Let's read in the data using `pandas` and get a quick look at it

In [ ]:
data = df
data.head()

Let's get the names of the wells from the data and assign them to a variable

In [ ]:
scaled_heights_colname = 'scaled_z'
data[scaled_heights_colname]  = data[DEPTH_FROM_AHD_COL] * 50

In [ ]:
data[['BoreID', scaled_heights_colname, DEPTH_FROM_AHD_COL]].head()

In [ ]:
site_ref_colname='name'
#data.index.values.astype(str)
data[site_ref_colname]=data.BoreID.values.astype(str)

In [ ]:
wells = data.name.unique()
# wells

Now we want to go through the data and build a `dict` for each well

In [ ]:
# split out the wells into a dictionary
well_dict = {}
for well in wells:
    well_dict["{0}".format(well)] = data[data.name == well]

Next let's take each well in the `well_dict` and get the points referenced to one another so they all plot up in real space relative to one another. We are then creating a `numpy` array with the `'easting'` (x-value), the `'northing'` (y-value), and the `depth` which is our true vertical depth. We now have each entry in the `points_dict` with an x, y, and z value.

In [ ]:
# now turn the wells into points
points_dict = {}
for points in well_dict:
    points_dict["{0}".format(points)] = np.array(
        list(
            zip(
                well_dict[points].Easting ,
                well_dict[points].Northing,
                well_dict[points][scaled_heights_colname],
            )
        )
    )

In [ ]:
well_dict[points]

In [ ]:
len(well_dict)

In [ ]:
points_dict[points]

From the points we want to create `PolyData` to plot in `PyVista`. To create this `PolyData` we use `PVGeo.points_to_poly_data()` on each entry in the `points_dict`. Then we use `PVGeo.filters.AddCellConnToPoints` to create cells connecting the points. At this point we have everything we need to plot up wellbore trajectories. But let's have some fun!

In [ ]:
# now turn points into polydata
lines_dict = {}
for lines in points_dict:
    poly = PVGeo.points_to_poly_data(points_dict[lines])
    lines_dict[
        "{0}".format(lines)
    ] = PVGeo.filters.AddCellConnToPoints().apply(poly)

In [ ]:
[x for x in lines_dict.keys()][0:6]

To color and size the `PolyData` by a value we use the `GR` column from the original dataset. We assigne the `GR` gamma-ray values to each `PolyData` object meaning we now have `DataArrays` for each well that we can use to color our wellbore. Finally, we use `.tube()` to create tubes at each point with radius 10 that scale with the `GR` gamma-ray values. 

In [ ]:
litho_class_col = 'Lithology_1_num'

In [ ]:
path = '10109370'
data[data[site_ref_colname] == path][litho_class_col].values

In [ ]:
lines_dict_tmp = {}

for path in lines_dict:
    pass

In [ ]:
path

In [ ]:
data[data[site_ref_colname] == path]

In [ ]:
litho_class_col

In [ ]:
path='10109370'

In [ ]:
vals = data[data[site_ref_colname] == path][litho_class_col].values
vals

In [ ]:
lines_dict[path]

In [ ]:
lines_dict[path]["GR"] = vals
lines_dict[path]

In [ ]:
lines_dict[path].tube(radius=10, scalars="GR", inplace=True)
if len(vals) > 1:
    lines_dict_tmp[path] = lines_dict[path]

In [ ]:
lines_dict_tmp = {}

In [ ]:
for path in lines_dict:
    vals = data[data[site_ref_colname] == path][litho_class_col].values
    lines_dict[path]["GR"] = vals
    lines_dict[path].tube(radius=10, scalars="GR", inplace=True)
    if len(vals) > 1:
        lines_dict_tmp[path] = lines_dict[path]

In [ ]:
len(vals)

In [ ]:
lines_dict[path]

In [ ]:
lines_dict = lines_dict_tmp

In [ ]:
len(lines_dict)

In [ ]:
path = '61470318'
lines_dict[path]

In [ ]:
path = '61400032'
lines_dict[path]
len(data[data[site_ref_colname] == path][litho_class_col].values)

We could plot up the wellbores at this point and have a great plot, but let's make some stratigraphic surfaces. I grabbed some tops from the [COGCC](https://cogcc.state.co.us) website for one of these wells. To make surfaces we need `x` and `y` values spanning the area we are interested in. So I use the minimum and maximum from the dataset and then add 1,000 feet in the x and y direction for the surfaces. Next we use `meshgrid` and create our grid. Then for each surface I create an empty array using `np.empty()` and `.fill` it with the depth to each top. For the last step for each surface we use `StructuredGrid` to create a structured `PolyData` grid. Here we have the Niobrara, Shannon, Sussex, Parkman, and Pierre formation tops.

In [ ]:
data.Easting.max() - data.Easting.min(),  (data.Northing.max() - data.Northing.min())

In [ ]:
x = np.arange(
    data.Easting.min() - 1000,
    data.Easting.max() + 1000,
    100,
)
y = np.arange(
    data.Northing.min() - 1000,
    data.Northing.max() + 1000,
    100,
)

In [ ]:
x.shape, y.shape

In [ ]:
x, y = np.meshgrid(x, y)

In [ ]:

# pie = np.empty(r.shape)
# pie.fill(-2267)
# pierregrid = pv.StructuredGrid(x, y, pie)

All that work and now we get so see how it turned out! We start by creating our `Plotter` and then `add_mesh` for each pice of `PolyData` that we have created so far. Lastly, we call `.show()` and we have an interactive set of wells!

In [ ]:
plotter = pv.Plotter()
for well in lines_dict:
    plotter.add_mesh(lines_dict[well])

#plotter.add_mesh(pierregrid, opacity=0.5, color="green")

plotter.show()

In [ ]:
well, lines_dict[well]

In [ ]:
# sphinx_gallery_thumbnail_number = 2
from pyvista import examples# sphinx_gallery_thumbnail_number = 2

In [ ]:
# Load St Helens DEM and warp the topography
mesh = examples.download_st_helens()
mesh

In [ ]:
type(mesh)

In [ ]:
mesh = mesh.warp_by_scalar()

In [ ]:
mesh

In [ ]:
type(mesh)

In [ ]:
# First a default plot with jet colormap
p = pv.Plotter()
# Add the data, use active scalar for coloring, and show the scalar bar
p.add_mesh(mesh)
# Display the scene
p.show()

In [ ]:
grid_res = dem_array_data['grid_res']
x_min, x_max, y_min, y_max = dem_array_data['bounds']
xx, yy = dem_array_data['mesh_xy']
dem_array = dem_array_data['dem_array']

In [ ]:
# mlab.figure(size=(800, 800))
# mlab.surf(xx, yy, dem_array, warp_scale=20, colormap='terrain')
# mlab.outline()
# mlab.show()

In [ ]:
grid = pv.StructuredGrid(xx, yy, dem_array * 20)

In [ ]:
grid

In [ ]:
type(grid)

In [ ]:
# TODO: this grid does not look like the one with the MT Saint Helen data set. How do I get the elevation values and a terrain coloring of voxels?? 

In [ ]:
grid.plot()

In [ ]:
# First a default plot with jet colormap
p = pv.Plotter()
# Add the data, use active scalar for coloring, and show the scalar bar
p.add_mesh(grid)
# Display the scene
p.show()

In [ ]:
# This is not at all clear what this is for. I tried to work by similarity possibly from 
# https://docs.pyvista.org/examples/00-load/create-structured-surface.html?highlight=plot_curvature

grid.plot_curvature(clim=[-1, 1])

## Appendix trial stuff

As expected frustrating to do something simple like an overlay in Python (yearning for R's ggmap). Google with keywords "overlay a rasterio with geopandas points".
For future reference perhaps https://geohackweek.github.io/vector/06-geopandas-advanced/  . 

https://gis.stackexchange.com/questions/294072/how-can-i-superimpose-a-geopandas-dataframe-on-a-raster-plot

We extract a portion of the dem over the bounding box of the groundwater areas of interest, and save DEM data as numpy arrays that will be more convenient to work with in mayavi (with x=easting and y=northing) 

In [ ]:
import pyvista
import PVGeo


In [ ]:
# This sets the plotting theme of `vtki` to look just like a ParaView rendering
#vtki.set_plot_theme('paraview')
#vtki.rcParams['cmap'] = 'bwr_r'